In [403]:
# import os
# import cv2
# path = "./DB4_B_2000"
# objects = os.listdir(path)
# files_file = [f for f in objects if os.path.isfile(os.path.join(path, f))]

In [404]:
# for file in files_file:
#     img = cv2.imread("./DB4_B_2000/"+file, 0)
#     img_resize = cv2.resize(img ,(288,384))
#     cv2.imwrite("./DB4_B_2000_resize/"+file, img_resize)

In [405]:
# import os
# import cv2
# path = "./DB4_B_2000_resize"
# objects = os.listdir(path)
# files_file = [f for f in objects if os.path.isfile(os.path.join(path, f))]

In [406]:
# list_shape_2000 = []
# for file in files_file:
#     img = cv2.imread("./DB4_B_2000_resize/"+file, 0)
#     list_shape_2000.append(img.shape)

In [407]:
# set(list_shape_2000)

In [408]:
# path = "./DB4_B_2004"
# objects = os.listdir(path)
# files_file = [f for f in objects if os.path.isfile(os.path.join(path, f))]

In [409]:
# list_shape_2004 = []
# for file in files_file:
#     img = cv2.imread("./DB4_B_2004/"+file, 0)
#     list_shape_2004.append(img.shape)

In [410]:
# set(list_shape_2004)

In [411]:
import cv2
import fingerprint_enhancer
import fingerprint_feature_extractor
import numpy as np
from projectname.matching1 import extract_angle, extract_tuple_profile, generate_tuple_profile, match_tuples, match
import json

In [412]:
img1 = cv2.imread('./data_unrename/101_1_DB4_B_2000.png',0)
img2 = cv2.imread('./data_unrename/101_2_DB4_B_2000.png',0)

In [413]:
out1 = fingerprint_enhancer.enhance_Fingerprint(img1)
out2 = fingerprint_enhancer.enhance_Fingerprint(img2)

In [414]:

FeaturesTerm1, FeaturesBif1 = fingerprint_feature_extractor.extract_minutiae_features(out1, showResult= False)
FeaturesTerm2, FeaturesBif2 = fingerprint_feature_extractor.extract_minutiae_features(out2, showResult= False)

In [415]:
len1 = FeaturesTerm1 + FeaturesBif1
len2 = FeaturesTerm2 + FeaturesBif2

In [416]:
location_minutiae1 = []
for i in range(len(len1)):
    location_minutiae1.append((len1[i].locX, len1[i].locY))

location_minutiae_bifurcation1 = []
location_minutiae_termination1 = []

for j in range(len(FeaturesTerm1)):
    location_minutiae_termination1.append((FeaturesTerm1[j].locX, FeaturesTerm1[j].locY))

for k in range(len(FeaturesBif1)):
    location_minutiae_bifurcation1.append((FeaturesBif1[k].locX, FeaturesBif1[k].locY))

In [417]:
location_minutiae2 = []
for i in range(len(len2)):
    location_minutiae2.append((len2[i].locX, len2[i].locY))

location_minutiae_bifurcation2 = []
location_minutiae_termination2 = []

for j in range(len(FeaturesTerm2)):
    location_minutiae_termination2.append((FeaturesTerm2[j].locX, FeaturesTerm2[j].locY))

for k in range(len(FeaturesBif2)):
    location_minutiae_bifurcation2.append((FeaturesBif2[k].locX, FeaturesBif2[k].locY))

In [418]:
feature_termination1 = generate_tuple_profile(location_minutiae1,location_minutiae_termination1,5)
feature_bifurcation1 = generate_tuple_profile(location_minutiae1,location_minutiae_bifurcation1,5)

feature1 = dict()
feature1['Termination']  = feature_termination1
feature1['Bifurcation']  = feature_bifurcation1

In [419]:
feature_termination2 = generate_tuple_profile(location_minutiae2,location_minutiae_termination2,5)
feature_bifurcation2 = generate_tuple_profile(location_minutiae2,location_minutiae_bifurcation2,5)

feature2 = dict()
feature2['Termination']  = feature_termination2
feature2['Bifurcation']  = feature_bifurcation2

In [420]:
termination_base = list(feature1.values())[0]
bifurcation_base = list(feature1.values())[1]
termination_test = list(feature2.values())[0]
bifurcation_test = list(feature2.values())[1]


In [421]:
def match_two_point(ratios_base, angles_base, ratios_test, angles_test, th_range, th_angle):
    count = 0
    list_done = []
    for i in range(len(ratios_base)):
        for j in range(len(ratios_base)):
            check = True
            if j not in list_done:
                if (((ratios_test[i] >= ratios_base[j] - th_range)  * (ratios_test[i] <= ratios_base[j] + th_range)) * ((angles_test[i] >= angles_base[j] - th_angle)  * (angles_test[i] <= angles_base[j] + th_angle))) == True:
                    count +=1
                    list_done.append(j)
                    check = False
                    break
            if check==False:
                break
    return count

In [422]:
ratios_test = np.array([ratios for c, [ratios, _] in termination_test.items()])
angles_test = np.array([angles for c, [_, angles] in termination_test.items()])
point_test = np.array([c for c, [_, angles] in termination_test.items()])
num_point_test = len(point_test)

ratios_base = np.array([ratios for c, [ratios, _] in termination_base.items()])
angles_base = np.array([angles for c, [_, angles] in termination_base.items()])
point_base = np.array([angles for c, [_, angles] in termination_base.items()])
num_point_base = len(point_base)

In [423]:
print(len(ratios_base), len(ratios_test))

29 25


In [424]:
th_range = 0.25
th_angle = 5

In [425]:
common_points_base = []
common_points_test = []

matrix_score_all_point= []
list_score_all_point= []

for i in range(num_point_test):
    list_score = []
    for j in range(num_point_base):
     
        # list_score_all_point.append(match_two_point(ratios_base[j],angles_base[j],ratios_test[i],angles_test[i],th_range, th_angle ))
        list_score.append(match_two_point(ratios_base[j],angles_base[j],ratios_test[i],angles_test[i],th_range, th_angle ))
    matrix_score_all_point.append(list_score)
index = min(num_point_base,num_point_test)
list_score_all_point =  np.array(matrix_score_all_point).reshape(-1)
list_sorted = sorted(list_score_all_point, reverse= True)

In [426]:
print(matrix_score_all_point)

[[4, 2, 2, 3, 3, 0, 2, 4, 1, 2, 4, 4, 1, 1, 1, 3, 1, 2, 2, 0, 1, 6, 2, 1, 2, 1, 2, 2, 4], [2, 0, 2, 3, 2, 1, 2, 1, 0, 2, 4, 1, 0, 0, 0, 1, 0, 3, 1, 3, 2, 2, 1, 1, 5, 2, 1, 1, 2], [2, 2, 2, 0, 3, 0, 2, 5, 2, 1, 2, 1, 1, 1, 0, 3, 1, 3, 5, 1, 3, 4, 4, 3, 2, 1, 5, 3, 3], [3, 3, 3, 2, 3, 2, 3, 5, 1, 1, 2, 2, 3, 2, 3, 4, 2, 3, 5, 1, 4, 3, 6, 3, 2, 1, 2, 3, 3], [2, 3, 3, 1, 2, 1, 1, 3, 1, 0, 1, 3, 3, 4, 1, 1, 3, 1, 2, 1, 1, 2, 0, 2, 2, 1, 1, 2, 0], [3, 3, 3, 1, 1, 0, 3, 0, 0, 2, 2, 3, 2, 4, 1, 2, 4, 0, 0, 1, 2, 1, 0, 1, 5, 2, 2, 2, 1], [2, 3, 1, 3, 1, 1, 0, 0, 1, 4, 2, 3, 3, 5, 1, 2, 2, 2, 1, 1, 1, 1, 0, 0, 2, 4, 1, 1, 0], [4, 4, 1, 1, 2, 1, 1, 3, 1, 0, 2, 2, 2, 3, 0, 2, 3, 2, 3, 0, 1, 4, 2, 4, 3, 1, 4, 2, 1], [3, 1, 2, 1, 3, 1, 4, 1, 0, 2, 2, 1, 2, 2, 1, 1, 3, 0, 3, 1, 3, 2, 1, 3, 3, 0, 1, 3, 2], [2, 3, 0, 3, 0, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 3, 1, 2, 4, 2], [3, 3, 2, 4, 2, 1, 2, 1, 1, 1, 4, 0, 1, 1, 1, 4, 2, 1, 2, 3, 4, 3, 2, 4, 4, 1, 3, 2, 6], [0, 4, 2, 1, 1, 3, 0

In [427]:
len(list_sorted)

725

In [428]:
common_points_base = []
common_points_test = []

In [429]:
list_point_done_row = []
list_point_done_column = []

T = len(ratios_test[0])

for k in range(index):
    
    for i in range(num_point_test):
        check1= True
        if i not in list_point_done_row:
            for j in range(num_point_base):
                check2 = True
                if j not in list_point_done_column and i not in list_point_done_row:
                    if matrix_score_all_point[i][j] ==list_sorted[0]:
                        

                        num_column = num_point_base
                        num_row = num_point_test

                        for x in range(num_column):
                            if x not in list_point_done_column:
                                list_sorted.remove(matrix_score_all_point[i][x])
                        print(len(list_sorted))
                        for y in range(num_row):
                            if y == i:
                                pass
                            else:
                                if y not in list_point_done_row:
                                    # print(matrix_score_all_point[y][j])
                                    list_sorted.remove(matrix_score_all_point[y][j])
                            
                                # print(list_sorted)
                               
                        list_point_done_row.append(i)
                        list_point_done_column.append(j)
                        if matrix_score_all_point[i][j] >= (T//2):
                            common_points_base.append(point_base[j])
                            common_points_test.append(point_test[i])
                        check1 =False
                        check2 = False
                if check2 == False:
                    break
        if check1 == False:
            break
                

696
644
594
546
500
456
414
374
336
300
266
234
204
176
150
126
104
84
66
50
36
24
14
6
0


In [430]:
list_point_done_row

[24,
 0,
 3,
 10,
 18,
 20,
 22,
 1,
 2,
 6,
 11,
 14,
 15,
 16,
 17,
 21,
 23,
 5,
 7,
 4,
 8,
 9,
 12,
 13,
 19]

In [431]:
list_point_done_column

[14,
 21,
 22,
 28,
 2,
 4,
 15,
 24,
 7,
 13,
 11,
 5,
 27,
 9,
 10,
 19,
 6,
 16,
 0,
 1,
 18,
 3,
 20,
 8,
 26]

In [432]:
# a,b = match(feature1,feature2,0.25,5)